<a href="https://colab.research.google.com/github/hhl13/House-price-predction/blob/main/House%20Price%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import the libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#importing the libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load the Housing.csv file
file_path = '/content/Housing.csv'

# Read the CSV file using pandas
data=pd.read_csv(file_path, encoding='ascii')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Housing.csv'

#General insights


In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.shape[:2]

In [ ]:
data.head(5)

In [ ]:
plt.hist(data['Overall Qual'])
plt.show()


In [ ]:
data['Lot Area'].median()


In [ ]:
data.tail(5)

In [ ]:

data['Overall Qual'].value_counts()


# Drop nan value

In [ ]:
data = data.drop(['Alley','Fireplace Qu','Pool QC','Fence','Misc Feature','Order','PID'], axis = 1)
data.dropna(inplace=True)
data.info()

In [ ]:
data.head(2)

In [ ]:
data.tail(2)

In [ ]:
data.info()

In [ ]:
# Split the data into categorical and numerical datasets
categorical_data = data.select_dtypes(include=['object'])
numerical_data = data.select_dtypes(exclude=['object'])
categorical_data.head(5)

In [ ]:
categorical_data.tail(2)

In [ ]:
numerical_data.head(2)

In [ ]:
numerical_data.tail(2)


Process numerical dataset

In [ ]:
# prompt: Using pearson correlation features selection for numerical data set

# Calculate the Pearson correlation matrix
corr_matrix = numerical_data.corr()

# Identify the features with high correlation (above 0.75)
correlated_features = corr_matrix[corr_matrix['SalePrice'] > 0.75].index

# Drop the correlated features from the numerical dataset
numerical_data = numerical_data.drop(correlated_features, axis=1)

#Add the SalePrice column again
numerical_data = pd.concat([numerical_data, data['SalePrice']], axis=1)

In [ ]:
numerical_data.head(5)

In [ ]:
from matplotlib import pyplot as plt
data['Lot Frontage'].plot(kind='line', figsize=(8, 4), title='Lot Frontage')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
import pandas as pd
from scipy.stats import f_oneway
from tqdm import tqdm

tqdm.pandas()


# Split the data into categorical and numerical datasets
categorical_data = data.select_dtypes(include=['object'])
numerical_data = data.select_dtypes(exclude=['object'])

# Perform ANOVA correlation coefficient feature selection for categorical data
anova_results = {}
for column in tqdm(categorical_data.columns):
    unique_values = categorical_data[column].unique()
    if len(unique_values) > 1:  # Ensure there are at least two unique values
        groups = [data[data[column] == value]['SalePrice'].values for value in unique_values]
        anova_results[column] = f_oneway(*groups)[1]  # p-value

# Filter out the strongly associated categorical features (p-value < 0.05)
strong_association = {k: v for k, v in anova_results.items() if v < 0.05}

# Keep only strongly associated categorical features
strongly_associated_categorical_data = categorical_data[list(strong_association.keys())]

# Output the strongly associated categorical features
print('Strongly associated categorical features with Sale Price:')
print(list(strong_association.keys()))

In [ ]:
categorical_data.info()

In [ ]:
onehot_data = pd.get_dummies(categorical_data, columns=categorical_data.columns)

In [ ]:
onehot_data

In [ ]:
combined_data = pd.concat([onehot_data, numerical_data], axis=1)
combined_data

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Define inputs and target variable
X = combined_data.drop('SalePrice', axis=1)
y = combined_data['SalePrice']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Fit the model on the training data
xgb_model.fit(X_train, y_train)

# Predict the SalePrice on the testing data
y_pred = xgb_model.predict(X_test)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)

# Output the R-squared value
print('R-squared value:', r2)




In [ ]:
pd.DataFrame({'Actual Price': y_test, 'Predicted Price': y_pred, 'Residuals': y_pred-y_test})


In [ ]:
# 5 most features importance and its importance values

feature_importances = xgb_model.feature_importances_
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
feature_importances.sort_values('importance', ascending=False).head(5)


In [ ]:
data['SalePrice'].plot(kind='hist', title='Distribution of Sale Prices')

In [ ]:
combined_data['Overall Qual']

In [ ]:
# Plotting distributions of a few numerical columns
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
#Distribution of Lot Area
sns.histplot(data['Lot Area'], bins=30, kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Distribution of Lot Area')
axes[0, 0].set_xlim(0, 50000) # Limiting x-axis for better readability

#Distribution of Year Built
sns.histplot(data['Year Built'], bins=30, kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Distribution of Year Built')

#Distribution of Overall Quality
sns.histplot(data['Overall Qual'], bins=10, kde=False, ax=axes[1, 0])
axes[1, 0].set_title('Distribution of Overall Quality')

#Distribution of Sale Price
sns.histplot(data['SalePrice'], bins=30, kde=True, ax=axes[1, 1])
axes[1, 1].set_title('Distribution of Sale Price')
axes[1, 1].set_xlim(0, 500000)

plt.tight_layout()
plt.show()
